In [1]:
from IPython.display import clear_output
import cv2
from pyzbar.pyzbar import decode
import zxing
import numpy as np
import subprocess
from PIL import Image
import IPython.display as display
import os
import time

In [2]:
def adjust_contrast(image, alpha=1.5, beta=0):
    """
    画像のコントラストを調整する関数。

    Args:
        image (numpy.ndarray): 入力画像
        alpha (float): コントラストの係数（1.0がデフォルトで、増やすほどコントラストが上がる）
        beta (int): 明るさ調整のオフセット値（負の値で暗くし、正の値で明るくする）

    Returns:
        numpy.ndarray: コントラスト調整後の画像
    """
    # 画像のコントラストと明るさを調整
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted


def show_image_with_replace(image):
    """
    画像を表示する際に、前の表示をクリアしてから新しい画像を表示する。

    Args:
        image (numpy.ndarray): 表示する画像。
    """
    # 画像を表示する前にログ上の出力をクリア
    clear_output(wait=True)

    # 新しい画像を表示
    cv2_imshow(image)




def PerspectiveTransform_2(image, W, H, top_left, top_right, bottom_right, bottom_left):
    # グレースケールに変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # コントラストを調整
    gray = adjust_contrast(gray, alpha=2.0, beta=50)

    # ノイズを軽減するためにガウシアンブラーを適用
    #gray_blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    Top_right = top_left
    Top_left = top_right
    Bottom_right = bottom_left
    Bottom_left = bottom_right

    pts1 = np.float32([Top_left, Top_right, Bottom_left, Bottom_right])

    # QRコードの取得精度に関わる
    width, height = W, H  # 変換後の画像サイズ

    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

    # 透視変換行列を計算
    matrix = cv2.getPerspectiveTransform(pts1, pts2)

    # 透視変換を適用して平面を補正
    warped = cv2.warpPerspective(gray, matrix, (width, height))
    return warped



def detect_qr_codes(image):
    """
    画像からQRコードを検出し、QRコードのデータと中心座標を返す関数。
    また、画像にQRコードの位置を描画して返します。

    Args:
        image (numpy.ndarray): QRコードを検出するための入力画像。

    Returns:
        list: 検出されたQRコードのデータと中心座標のリスト。
              各要素は辞書で、データ (data) と中心座標 (center) を含む。
        numpy.ndarray: QRコードの位置が描画された画像。
    """
    # QRコードを検出
    reader = zxing.BarCodeReader()
    decoded_objects = reader.decode(image)
    # decoded_objects = decode(image)

    # 検出されたQRコードの情報を格納するリスト
    qr_info = []

    # QRコードが検出されたかどうかを確認
    if len(decoded_objects) > 0:
        for obj in decoded_objects:
            # QRコードのバウンディングボックスの座標を取得
            points = obj.polygon

            # バウンディングボックスの座標を表示
            if len(points) == 4:
                # 四角形の四隅の座標
                n_points = len(points)
                for j in range(n_points):
                    pt1 = (int(points[j][0]), int(points[j][1]))
                    pt2 = (int(points[(j + 1) % n_points][0]), int(points[(j + 1) % n_points][1]))
                    cv2.line(image, pt1, pt2, (0, 255, 0), 3)

                # QRコードの中心点を計算
                center_x = int((points[0][0] + points[2][0]) / 2)
                center_y = int((points[0][1] + points[2][1]) / 2)
                cv2.circle(image, (center_x, center_y), 5, (255, 0, 0), -1)

                # QRコードのデータを取得
                qr_data = obj.data.decode("utf-8")
                print(f"QRコードのデータ: {qr_data}")
                print(f"QRコードの中心座標: ({center_x}, {center_y})")

                # QRコード情報をリストに追加
                qr_info.append({
                    "data": qr_data,
                    "center": (center_x, center_y)
                })

    return qr_info, image


def get_qr_grid_positions(image,qr_info, grid_size, cell_width, cell_height):
    """
    QRコードの中心座標がグリッドのどのマスに存在するかを判定する関数。
    また、各マスの情報を全て返し、QRコードが存在する場合にはその情報も追加。

    Args:
        qr_info (list): QRコードのデータと中心座標のリスト。
        grid_size (int): グリッドのサイズ。
        cell_width (int): 各グリッドの幅。
        cell_height (int): 各グリッドの高さ。

    Returns:
        list: すべてのマス目の情報を格納したリスト。
              QRコードが存在するマス目にはその情報を追加。
    """
    grid_positions = []

    for y in range(grid_size):
        for x in range(grid_size):
            # マス目の情報
            grid_top_left = (int(x * cell_width), int(y * cell_height))
            grid_bottom_right = (int((x + 1) * cell_width), int((y + 1) * cell_height))

            # 初期状態のマス目の情報を作成
            cell_info = {
                "grid_xy": (x, y),
                "grid_top_left": grid_top_left,
                "grid_bottom_right": grid_bottom_right,
                "qr_data": None,
                "qr_center": None
            }

            # QRコードの情報を追加
            for qr in qr_info:
                center_x, center_y = qr['center']
                if grid_top_left[0] <= center_x < grid_bottom_right[0] and grid_top_left[1] <= center_y < grid_bottom_right[1]:
                    cell_info["qr_data"] = qr['data']
                    cell_info["qr_center"] = qr['center']
                    break

            # マス目の情報をリストに追加
            grid_positions.append(cell_info)

            #グリッドの境界線を描画
            cv2.rectangle(image, grid_top_left, grid_bottom_right, (0, 255, 0), 1)

            # グリッドの中心座標を計算
            center_x = (grid_top_left[0] + grid_bottom_right[0]) // 2
            center_y = (grid_top_left[1] + grid_bottom_right[1]) // 2

            # グリッドのXY座標を描画
            cv2.putText(image, f"{x},{y}", (center_x - 15, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)


    detected_grids = []
    for qr in qr_info:
        center_x, center_y = qr['center']

        # QRコードの中心座標からマス目のXY座標を計算
        grid_x = center_x // cell_width
        grid_y = center_y // cell_height

        # グリッドの座標を文字列形式でリストに追加
        grid_position = f"{int(grid_x)},{int(grid_y)}"
        detected_grids.append(grid_position)



    return grid_positions,detected_grids

def draw_grid_info(image, grid_positions):
    """
    グリッド情報を画像上に描画する関数。

    Args:
        image (numpy.ndarray): 画像
        grid_positions (list): 各マス目の情報を格納したリスト
    """
    for pos in grid_positions:
        # グリッドの中心座標を計算
        grid_x, grid_y = pos['grid_xy']
        top_left = pos['grid_top_left']
        bottom_right = pos['grid_bottom_right']
        center_x = (top_left[0] + bottom_right[0]) // 2
        center_y = (top_left[1] + bottom_right[1]) // 2

        # グリッドのXY座標を描画
        cv2.putText(image, f"{grid_x},{grid_y}", (center_x - 15, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)

        # QRコードが存在する場合、そのデータを描画
        if pos["qr_data"]:
            cv2.putText(image, f"QR: {pos['qr_data']}", (center_x - 15, center_y + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)


def get_grid_code(W, H, N, X, Y):
    """
    与えられた座標 (X, Y) が (W, H) サイズの画像を N x N のグリッドに分割したとき、
    どのグリッドに属するかを判定する関数。左上を (0, 0) とする。

    Args:
        W (int): 画像の横幅。
        H (int): 画像の高さ。
        N (int): グリッドの分割数（N x N グリッド）。
        X (int): 座標のX成分。
        Y (int): 座標のY成分。

    Returns:
        tuple: 座標 (X, Y) が属するグリッドの行番号と列番号 (row, col)。
    """
    # 各グリッドの幅と高さを計算
    grid_width = W / N
    grid_height = H / N

    # グリッドの列番号（X成分）と行番号（Y成分）を計算
    col = int(X // grid_width)
    row = int(Y // grid_height)


    # 行番号と列番号を返す（左上が (0, 0) の基準）
    return (col,row )

def create_virtual_grid(N, W, H, detected_grids, cell_width, cell_height):
    """
    仮想的なグリッドを作成し、QRコードが検出されたマス目を塗りつぶす。

    Args:
        N (int): グリッドの分割数。
        W (int): 仮想グリッドの横幅。
        H (int): 仮想グリッドの高さ。
        detected_grids (list): 検出されたQRコードが存在するグリッドの座標リスト。
        cell_width (int): 各グリッドの幅。
        cell_height (int): 各グリッドの高さ。

    Returns:
        numpy.ndarray: グリッドが描かれた新しい画像。
    """
    # 白いキャンバスを作成
    grid_image = np.ones((H, W, 3), dtype=np.uint8) * 255  # 255は白を意味する

    # グリッドを描画
    for y in range(N):
        for x in range(N):
            top_left = (int(x * cell_width), int(y * cell_height))
            bottom_right = (int((x + 1) * cell_width), int((y + 1) * cell_height))

            # 検出されたQRコードのグリッドならば赤で塗りつぶし、そうでなければ枠のみ
            if (x, y) in detected_grids:
                cv2.rectangle(grid_image, top_left, bottom_right, (0, 0, 255), -1)  # 赤色で塗りつぶし
            else:
                cv2.rectangle(grid_image, top_left, bottom_right, (0, 0, 0), 1)  # 黒色の枠線のみ

    return grid_image

def setting_sikaku():
    # リアルタイムでカメラ映像を表示
    print("任意のキーを押して写真を撮影してください。")
    while True:
        ret, frame = cap.read()

        if not ret:
            print("カメラから画像を取得できませんでした。")
            break

        # カメラ映像を表示
        cv2.imshow("Webcam - Press any key to capture", frame)

        # 'q'キーで撮影せずに終了
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("終了します。")
            cap.release()
            cv2.destroyAllWindows()
            exit()

        # 任意のキーを押すと撮影
        if cv2.waitKey(1) != -1:
            print("写真を撮影しました。")
            break

    # 矩形を選択
    roi = cv2.selectROI("Select ROI", frame, showCrosshair=True, fromCenter=False)
    cv2.destroyWindow("Select ROI")  # ROI選択後にウィンドウを閉じる

    # ROIの四隅の座標を取得
    x, y, w, h = roi

    # 四隅の座標を計算
    top_left = (x, y)
    top_right = (x + w, y)
    bottom_left = (x, y + h)
    bottom_right = (x + w, y + h)

    # 座標の表示
    print(f"左上: {top_left}")
    print(f"右上: {top_right}")
    print(f"左下: {bottom_left}")
    print(f"右下: {bottom_right}")

    # 四隅の座標に点を描画して確認
    cv2.circle(frame, top_left, 5, (0, 255, 0), -1)
    cv2.circle(frame, top_right, 5, (255, 0, 0), -1)
    cv2.circle(frame, bottom_left, 5, (0, 0, 255), -1)
    cv2.circle(frame, bottom_right, 5, (255, 255, 0), -1)

    # 結果を表示
    cv2.imshow("Selected ROI", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # リソースの解放
    cap.release()

    return top_left,top_right,bottom_right,bottom_left



def main(image,top_left,top_right,bottom_right,bottom_left):
    # 例: 画像サイズ 500x500 を 5x5 のグリッドに分割し、座標 (120, 220) が属するグリッドを判定
    W, H = 500, 500
    N = 5

    cell_width=W/N
    cell_height=H/N

    #grid_code = get_grid_code(W, H, N, X, Y)

    perce=PerspectiveTransform_2(image,W,H,top_left,top_right,bottom_right,bottom_left)


    qr_info, annotated_image = detect_qr_codes(perce)


    # QRコードの中心が存在するマス目のXY座標を取得し、各グリッドの情報をリストとして返す
    grid_positions,detected_grids = get_qr_grid_positions(image,qr_info, N, cell_width, cell_height)

    # グリッド情報を画像上に描画
    draw_grid_info(annotated_image, grid_positions)

    # QRコードが検出されたグリッド座標を表示
    #print("QRコードが検出されたグリッド座標:")

    grid_list=[]
    for i in range(len(qr_info)):
        col,row=get_grid_code(1000,1000,5,qr_info[i]["center"][0],qr_info[i]["center"][1])
        grid_list.append((col,row))

    print(grid_list)


In [8]:
#カメラの設定　デバイスIDは0
cap = cv2.VideoCapture(3)

#繰り返しのためのwhile文
while True:
    #カメラからの画像取得
   
    ret, frame = cap.read()

    #カメラの画像の出力
    cv2.imshow('camera' , frame)

    #繰り返し分から抜けるためのif文
    key =cv2.waitKey(10)
    if key == 27:
        break

#メモリを解放して終了するためのコマンド
cap.release()
cv2.destroyAllWindows()

In [10]:
# カメラの設定
cap = cv2.VideoCapture(3)  # デバイスIDはカメラに合わせて調整

# 任意のROI（矩形）を設定するための関数から座標を取得
top_left, top_right, bottom_right, bottom_left = setting_sikaku()

while True:
    # カメラからフレームを取得
    ret, frame = cap.read()
    if not ret:
        print("カメラから画像を取得できませんでした。")
        break

    main(frame,top_left, top_right, bottom_right, bottom_left)


    # 'ESC'キーで終了
    if cv2.waitKey(1) & 0xFF == 27:
        break

# カメラとウィンドウのリソース解放
cap.release()
cv2.destroyAllWindows()


任意のキーを押して写真を撮影してください。
写真を撮影しました。
左上: (0, 0)
右上: (0, 0)
左下: (0, 0)
右下: (0, 0)
カメラから画像を取得できませんでした。


In [26]:
from IPython.display import clear_output
# import cv2
# from pyzbar.pyzbar import decode
import time
# カメラの設定
cap = cv2.VideoCapture(3)
if not cap.isOpened():
    print("カメラが見つかりませんでした。")
    exit()

while True:
    # time.sleep(1)
    # カメラからフレームを取得
    ret, frame = cap.read()
    if not ret:
        print("カメラから画像を取得できませんでした。")
        break

    # QRコードを検出して情報を表示
    barcodes = decode(frame)
    for barcode in barcodes:
        # QRコードの位置を取得
        (x, y, w, h) = barcode.rect
        # QRコードの内容をデコード
        barcode_data = barcode.data.decode("utf-8")
        barcode_type = barcode.type

        # QRコードの位置を四角で囲む
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # QRコードの内容をフレーム上に表示
        text = f"{barcode_type}: {barcode_data}"
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # フレームを表示
    cv2.imshow("QR code scanner", frame)

    # 'ESC'キーで終了
    if cv2.waitKey(1) & 0xFF == 27:
        break

# カメラとウィンドウのリソース解放
cap.release()
cv2.destroyAllWindows()



In [19]:
import qrcode
from PIL import Image

In [20]:
def generate_qr_code(data, size_mm, file_path="qrcode.png"):
    # ミリメートルからピクセルへの変換（1インチ = 25.4mm、解像度 300 dpiと仮定）
    dpi = 300
    size_inch = size_mm / 25.4  # mmからインチに変換
    size_px = int(size_inch * dpi)  # ピクセルに変換

    # QRコード生成
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=2,
    )
    qr.add_data(data)
    qr.make(fit=True)

    # 生成したQRコードをPILイメージに変換
    img = qr.make_image(fill="black", back_color="white").convert('RGB')

    # 指定サイズにリサイズ
    img = img.resize((size_px, size_px), Image.LANCZOS)

    # QRコードをファイルに保存
    img.save(file_path)
    print(f"QRコードを{file_path}に保存しました。")

# 使い方
size_mm = 30  # QRコードのサイズを50mmに設定
for no in range(25):
    data = f"building_{no}"
    generate_qr_code(data, size_mm, f"../pictures/qr_{no}.png")


QRコードを../pictures/qr_0.pngに保存しました。
QRコードを../pictures/qr_1.pngに保存しました。
QRコードを../pictures/qr_2.pngに保存しました。
QRコードを../pictures/qr_3.pngに保存しました。
QRコードを../pictures/qr_4.pngに保存しました。
QRコードを../pictures/qr_5.pngに保存しました。
QRコードを../pictures/qr_6.pngに保存しました。
QRコードを../pictures/qr_7.pngに保存しました。
QRコードを../pictures/qr_8.pngに保存しました。
QRコードを../pictures/qr_9.pngに保存しました。
QRコードを../pictures/qr_10.pngに保存しました。
QRコードを../pictures/qr_11.pngに保存しました。
QRコードを../pictures/qr_12.pngに保存しました。
QRコードを../pictures/qr_13.pngに保存しました。
QRコードを../pictures/qr_14.pngに保存しました。
QRコードを../pictures/qr_15.pngに保存しました。
QRコードを../pictures/qr_16.pngに保存しました。
QRコードを../pictures/qr_17.pngに保存しました。
QRコードを../pictures/qr_18.pngに保存しました。
QRコードを../pictures/qr_19.pngに保存しました。
QRコードを../pictures/qr_20.pngに保存しました。
QRコードを../pictures/qr_21.pngに保存しました。
QRコードを../pictures/qr_22.pngに保存しました。
QRコードを../pictures/qr_23.pngに保存しました。
QRコードを../pictures/qr_24.pngに保存しました。
